# モデルの精度を確認するためのインターフェース

ver.2023/1/1

「Train_MMVC.ipynb」で学習したモデルでTTSと非リアルタイムのVCを行い、モデルの精度を検証します。

​

In [ ]:
#@title ## 0 ノートブックの準備

#@markdown このノートブックのセットアップを行います。セルを実行し、完了したら次に進んでください。

#debug用ディレクトリの作成
!rm -rf /mmvc-debug
!mkdir /mmvc-debug

#現在時刻の取得
import datetime
ｊst_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(jst_delta, 'JST')
now = datetime.datetime.now(JST)
nowt = now.strftime('%Y%m%d%H%M%S')

#python管理
import sys

#出力記録用カスタムマジック %%ccapture
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic
from IPython.utils.capture import capture_output

@magic_arguments.magic_arguments()
@magic_arguments.argument('output', type=str, default='', nargs='?')

@register_cell_magic
def ccapture(line, cell):
    args = magic_arguments.parse_argstring(ccapture, line)
    with capture_output() as outputs:
        get_ipython().run_cell(cell)
    if args.output:
        get_ipython().user_ns[args.output] = outputs
    
    outputs()

​

In [ ]:
%%ccapture one_mount_gdrive
print("----------------------------------------------------------------------------------------------------")
print("1 Google Driveをマウント")

#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
%%ccapture two_cd_mmvc_trainer
print("----------------------------------------------------------------------------------------------------")
print("2 MMVC_Trainerディレクトリに移動")

#@title ## 2 MMVC_Trainerディレクトリに移動
#@markdown ​マウントしたGoogle DriveのMMVC_Trainerディレクトリに移動します。

#@markdown Google DriveでMMVC_Trainerの場所を確認し、以下でパスを指定してください。

#@markdown 正しいパスが指定されていれば、以下のようなメッセージが表示されます。

#@markdown ```
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown ```
#@markdown


#@markdown ​
#@markdown ### Settings
directory = "/content/drive/MyDrive/MMVC_Trainer-main" #@param {type:"string"}

%cd $directory
!ls -1

​

In [ ]:
%%ccapture three_install_library
print("----------------------------------------------------------------------------------------------------")
print("3 ライブラリのインストール")

#@title ## 3 ライブラリのインストール
#@markdown 時間がかかります。気長にお待ちください。

!apt-get install espeak
!pip install -r requirements.txt --progress-bar off

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

​

In [ ]:
%%ccapture four_load_model
print("----------------------------------------------------------------------------------------------------")
print("4 学習したモデルの読み込み")

#@title ## 4 学習したモデルの読み込み
#@markdown 学習したモデルを読み込みます。以下のSettingsで必要な設定を行った後、セルを実行してください。

#@markdown ​
#@markdown ### Settings
#@markdown CONFIG_PATH：configファイルのパス
#@markdown 作成したconfigファイル(json)を指定してください。  
#@markdown `configs/****.json` のような値になります。
CONFIG_PATH = "configs/train_config.json" #@param {type:"string"}

#@markdown NET_PATH：学習したモデルのパス
#@markdown 作成されたモデル(pth)を指定してください。
#@markdown `logs/xxxxxxxx_xxxxx/G_xxxxx.pth` のような値になります。
NET_PATH = "logs/20220306_24000/G_xxxxx.pth"  #@param {type:"string"}

hps = utils.get_hparams_from_file(CONFIG_PATH)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint(NET_PATH, net_g, None)

​

In [ ]:
%%ccapture five_test_vc
print("----------------------------------------------------------------------------------------------------")
print("5 学習したモデルで非リアルタイムVCを行う")

#@title ## 5 学習したモデルで非リアルタイムVCを行う
#@markdown 非リアルタイムのVCを行います。以下のSettingsで必要な設定を行った後、セルを実行してください。

#@markdown 性能が悪い場合、学習不足か他に問題があります。

#@markdown ​
#@markdown ### Settings
#@markdown SOURCE_SPEAKER_ID
#@markdown ソース話者のID
SOURCE_SPEAKER_ID = 107 #@param {type:"integer"}

#@markdown SOURCE_WAVFILE
#@markdown ソース話者の音声ファイルのパス
#@markdown 学習に使用した音声ファイルか、新規に録音した音声ファイルを指定してください。
SOURCE_WAVFILE = "dataset/textful/00_myvoice/wav/VOICEACTRESS100_001.wav" #@param {type:"string"}

#@markdown TARGET_ID
#@markdown ターゲット話者のID
TARGET_ID = 100 #@param {type:"integer"}

with torch.no_grad():
    dataset = TextAudioSpeakerLoader(hps.data.validation_files_notext, hps.data)
    data = dataset.get_audio_text_speaker_pair([SOURCE_WAVFILE, SOURCE_SPEAKER_ID, "a"])
    data = TextAudioSpeakerCollate()([data])
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x for x in data]
    sid_tgt1 = torch.LongTensor([TARGET_ID])
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate))

​

## サポート専用

**以下はお問い合わせの際、指示があった場合のみ使用してください。**

In [ ]:
#@markdown **このセルは無視してください。**

#@markdown このセルは、セルが一括で実行されることを防ぐためのものです。

#@markdown  実行してしまった場合は、左側のアイコンをクリックしてセルを終了してください。

#一括実行の阻止
import time
time.sleep(86400)

​

In [ ]:
#@title ## サポート用ファイルの作成

#@markdown セルを実行すると内部処理が行われ、zipファイルが操作中のPC(またはタブレットなど)にダウンロードされます。

#@markdown ダウンロードされるzipファイルには、以下のファイルや情報が含まれます。

#@markdown * MMVC_Trainerフォルダ内のファイルの一覧
#@markdown * このノートブックで使用されている変数(自動的に設定されるものと、ユーザーが入力するものがあります)のリスト
#@markdown * このセッション内の出力
#@markdown * configsフォルダ内及びlogsフォルダ内のファイル

#@markdown これらには、ユーザーの個人情報が含まれる可能性があります。ダウンロード完了後、ファイルを共有する前に、必ず内容をご確認ください。

#@markdown ファイルのダウンロードが完了したら、ランタイムを切断してください。

#ファイルの準備
variable_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-variable.txt"
tree_dic_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-tree_dic.txt"
export_zip = "/mmvc-debug-" + str(nowt)
export_zipp = "/mmvc-debug-" + str(nowt) + ".zip"

#変数の値を保存
#whos使うと長い文字列が省略されるため、変数毎に取得する
##変数の一覧を取得
vlist = %who_ls
##それぞれの変数で値を取得してファイルに保存
with open(variable_txt, 'w') as f:
  for ev in vlist:
    try:
      #変数名(str)
      print(ev, end=' : ', file=f) 
      #変数の型(変数名がstrとなっているためevalでkeyに直す)
      print(type(eval(ev)), end=' : ', file=f)
      #変数の内容(変数名がstrとなっているためevalでkeyに直す)
      print(eval(ev), file=f)
    except:
      pass

#tree
!apt install tree
##ディレクトリ内以下
import traceback
with open(tree_dic_txt, 'w') as f:
  try:
    !tree {directory} > {tree_dic_txt}
  except Exception as e:
    print("An error occurred!", file=f)
    print(e, file=f)
    print(traceback.print_exc(), file=f)

#logsとconfigsの保存
#directoryが無かったらmkdirしてno-dic.txt置く
!if [ -d {directory}/logs ]; then if [ -z "$(ls {directory}/logs)" ]; then touch {directory}/logs/no-file.txt;else cp -rp {directory}/logs /mmvc-debug/logs; fi;else mkdir /mmvc-debug/logs && touch /mmvc-debug/logs/no-dic.txt; fi
!if [ -d {directory}/configs ]; then if [ -z "$(ls {directory}/configs)" ]; then touch {directory}/configs/no-file.txt;else cp -rp {directory}/configs /mmvc-debug/configs; fi;else mkdir /mmvc-debug/configs && touch /mmvc-debug/configs/no-dic.txt; fi

#直近のtracebackの保存
with open('/mmvc-debug/traceback.txt', 'w') as f:
  try:
    print(sys.last_type, sys.last_value, sys.last_traceback, file=f)
  except:
    pass

#ccaptureの保存
!mkdir /mmvc-debug/ccapture
with open('/mmvc-debug/ccapture/one_mount_gdrive.txt', 'w') as f:
  try:
    print(one_mount_gdrive, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/two_cd_mmvc_trainer.txt', 'w') as f:
  try:
    print(two_cd_mmvc_trainer, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/three_install_library.txt', 'w') as f:
  try:
    print(three_install_library, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/four_load_model.txt', 'w') as f:
  try:
    print(four_load_model, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/five_test_vc.txt', 'w') as f:
  try:
    print(five_test_vc, file=f)
  except:
    pass

#zipにまとめる
!apt install zip
!zip {export_zip} -r /mmvc-debug

#colabのfilesモジュールを使ってダウンロード
from google.colab import files
files.download(export_zipp)